# Import Libraries

In [2]:
import pandas as pd
import numpy as np
import optuna
import re
import lightgbm as lgb
from IPython.display import FileLink
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder, RobustScaler
from datetime import datetime
from optuna.samplers import TPESampler
import warnings
warnings.filterwarnings('ignore')

# Load Train and Test Datasets

In [4]:
# Load training and test datasets
train_df = pd.read_csv("C:/Users/SHARON/Downloads/Train (1).csv")
test_df = pd.read_csv("C:/Users/SHARON/Downloads/Test (1).csv")
train_df.head()

,ID,customer_id,country_id,tbl_loan_id,lender_id,loan_type,Total_Amount,Total_Amount_to_Repay,disbursement_date,due_date,duration,New_versus_Repeat,Amount_Funded_By_Lender,Lender_portion_Funded,Lender_portion_to_be_repaid,target
0,ID_266671248032267278,266671,Kenya,248032,267278,Type_1,8448.0,8448.0,2022-08-30,2022-09-06,7,Repeat Loan,120.85,0.014305,121.0,0
1,ID_248919228515267278,248919,Kenya,228515,267278,Type_1,25895.0,25979.0,2022-07-30,2022-08-06,7,Repeat Loan,7768.50,0.300000,7794.0,0
2,ID_308486370501251804,308486,Kenya,370501,251804,Type_7,6900.0,7142.0,2024-09-06,2024-09-13,7,Repeat Loan,1380.00,0.200000,1428.0,0
3,ID_266004285009267278,266004,Kenya,285009,267278,Type_1,8958.0,9233.0,2022-10-20,2022-10-27,7,Repeat Loan,2687.40,0.300000,2770.0,0
4,ID_253803305312267278,253803,Kenya,305312,267278,Type_1,4564.0,4728.0,2022-11-28,2022-12-05,7,Repeat Loan,1369.20,0.300000,1418.0,0


In [5]:
test_df.head()

,ID,customer_id,country_id,tbl_loan_id,lender_id,loan_type,Total_Amount,Total_Amount_to_Repay,disbursement_date,due_date,duration,New_versus_Repeat,Amount_Funded_By_Lender,Lender_portion_Funded,Lender_portion_to_be_repaid
0,ID_269404226088267278,269404,Kenya,226088,267278,Type_1,1919.0,1989.0,2022-07-27,2022-08-03,7,Repeat Loan,575.7,0.300000,597.0
1,ID_255356300042267278,255356,Kenya,300042,267278,Type_1,2138.0,2153.0,2022-11-16,2022-11-23,7,Repeat Loan,0.0,0.000000,0.0
2,ID_257026243764267278,257026,Kenya,243764,267278,Type_1,8254.0,8304.0,2022-08-24,2022-08-31,7,Repeat Loan,207.0,0.025079,208.0
3,ID_264617299409267278,264617,Kenya,299409,267278,Type_1,3379.0,3379.0,2022-11-15,2022-11-22,7,Repeat Loan,1013.7,0.300000,1014.0
4,ID_247613296713267278,247613,Kenya,296713,267278,Type_1,120.0,120.0,2022-11-10,2022-11-17,7,Repeat Loan,36.0,0.300000,36.0


# Combine Train and Test Data for Encoding and Define Features


In [7]:
# Combine train and test data for encoding
combined_df = pd.concat([train_df, test_df], axis=0)

categorical_cols = ['country_id', 'loan_type', 'New_versus_Repeat']
target_col = 'target'

# 1. Convert categorical columns to numerical values using LabelEncoder
label_encoder = LabelEncoder()

# Ensure that all categorical columns are properly encoded in the combined DataFrame
for col in categorical_cols:
    combined_df[col] = label_encoder.fit_transform(combined_df[col])

# Split the combined DataFrame back into train and test sets
train_df[categorical_cols] = combined_df[categorical_cols].iloc[:len(train_df)]
test_df[categorical_cols] = combined_df[categorical_cols].iloc[len(train_df):]

# 2. Define features (X) and target (y) for training
X = train_df[categorical_cols]
y = train_df[target_col]

# Check if all categorical columns are now numeric
print(X.dtypes)

country_id           int32
loan_type            int32
New_versus_Repeat    int32
dtype: object


# Process Date Columns and Extract Date Features

In [9]:
# Handle date columns
date_cols = ['disbursement_date', 'due_date']
for col in date_cols:
    train_df[col] = pd.to_datetime(train_df[col])
    test_df[col] = pd.to_datetime(test_df[col])
    
   # Create new features from date columns
    train_df[f'{col}_year'] = train_df[col].dt.year
    train_df[f'{col}_month'] = train_df[col].dt.month
    train_df[f'{col}_day'] = train_df[col].dt.day
    
    test_df[f'{col}_year'] = test_df[col].dt.year
    test_df[f'{col}_month'] = test_df[col].dt.month
    test_df[f'{col}_day'] = test_df[col].dt.day
# Drop original date columns
train_df = train_df.drop(columns=date_cols)
test_df = test_df.drop(columns=date_cols)

# Data Cleaning

In [11]:
def clean_column_names(train_df):
    # Replace all non-alphanumeric characters (except underscores) with underscores
    train_df.columns = [re.sub(r'\W+', '_', col) for col in train_df.columns]
    return train_df

def clean_column_names(test_df):
    # Replace all non-alphanumeric characters (except underscores) with underscores
    test_df.columns = [re.sub(r'\W+', '_', col) for col in test_df.columns]
    return test_df

In [12]:
print("Columns in train_df:", train_df.columns.tolist()) 
print("Columns in test_df:", test_df.columns.tolist())

Columns in train_df: ['ID', 'customer_id', 'country_id', 'tbl_loan_id', 'lender_id', 'loan_type', 'Total_Amount', 'Total_Amount_to_Repay', 'duration', 'New_versus_Repeat', 'Amount_Funded_By_Lender', 'Lender_portion_Funded', 'Lender_portion_to_be_repaid', 'target', 'disbursement_date_year', 'disbursement_date_month', 'disbursement_date_day', 'due_date_year', 'due_date_month', 'due_date_day']
Columns in test_df: ['ID', 'customer_id', 'country_id', 'tbl_loan_id', 'lender_id', 'loan_type', 'Total_Amount', 'Total_Amount_to_Repay', 'duration', 'New_versus_Repeat', 'Amount_Funded_By_Lender', 'Lender_portion_Funded', 'Lender_portion_to_be_repaid', 'disbursement_date_year', 'disbursement_date_month', 'disbursement_date_day', 'due_date_year', 'due_date_month', 'due_date_day']


In [13]:
#checking null values 
train_df.isnull().sum()
test_df.isnull().sum()

ID                             0
customer_id                    0
country_id                     0
tbl_loan_id                    0
lender_id                      0
loan_type                      0
Total_Amount                   0
Total_Amount_to_Repay          0
duration                       0
New_versus_Repeat              0
Amount_Funded_By_Lender        0
Lender_portion_Funded          0
Lender_portion_to_be_repaid    0
disbursement_date_year         0
disbursement_date_month        0
disbursement_date_day          0
due_date_year                  0
due_date_month                 0
due_date_day                   0
dtype: int64

In [14]:
train_df.shape
test_df.shape

(18594, 19)

In [15]:
train_df.duplicated().sum()
test_df.duplicated().sum()

0

# Feature Engineering 


In [17]:

# Assuming economic_indicators is a DataFrame containing additional data
economic_indicators = pd.read_csv("C:/Users/SHARON/Downloads/economic_indicators.csv")

economic_indicators_long = pd.melt(economic_indicators, id_vars=['Country', 'Indicator'],
                                   var_name='Year', value_name='Value')
economic_indicators_long['Year'] = economic_indicators_long['Year'].str.extract('(\d{4})').astype(int)
economic_indicators_pivot = economic_indicators_long.pivot_table(index=['Country', 'Year'],
                                                                 columns='Indicator',
                                                                 values='Value',
                                                                 aggfunc='first').reset_index()
economic_indicators_pivot = economic_indicators_pivot.rename(columns={'Country': 'country_id'})
# Drop rows with any missing values
economic_indicators_pivot.dropna(inplace=True)

missing_values = economic_indicators_pivot.isnull().sum() 
print("Missing values in each column:\n", missing_values)

Missing values in each column:
 Indicator
country_id                                                   0
Year                                                         0
Average precipitation in depth (mm per year)                 0
Deposit interest rate (%)                                    0
Fossil fuel energy consumption (% of total)                  0
Inflation, consumer prices (annual %)                        0
Interest rate spread (lending rate minus deposit rate, %)    0
Lending interest rate (%)                                    0
Official exchange rate (LCU per US$, period average)         0
Real interest rate (%)                                       0
Unemployment rate                                            0
dtype: int64


In [18]:
train_df['country_id'] = train_df['country_id'].astype(str)
test_df['country_id'] = test_df['country_id'].astype(str) 
economic_indicators_pivot['country_id'] = economic_indicators_pivot['country_id'].astype(str)
print(train_df['country_id'].dtype)  # Should output: 'object' or 'str'
print(test_df['country_id'].dtype)   # Should output: 'object' or 'str'
print(economic_indicators_pivot['country_id'].dtype) 

object
object
object


# Merge Economic Indicators with Training and Testing Data, Clean and Handle Missing Values

In [20]:
# Extract year from disbursement_date and merge economic indicators with train and test datasets
train_eco_merge = pd.merge(train_df, economic_indicators_pivot,
                    left_on=['country_id', 'disbursement_date_year'], 
                    right_on=['country_id', 'Year'], how='left').drop(columns=['Year'])
test_eco_merge = pd.merge(train_df, economic_indicators_pivot,
                    left_on=['country_id', 'disbursement_date_year'], 
                    right_on=['country_id', 'Year'], how='left').drop(columns=['Year'])

# Ensure no duplicated columns in the merged dataframes before concatenation
train_eco_merge = train_eco_merge.loc[:, ~train_eco_merge.columns.duplicated()]
test_eco_merge = test_eco_merge.loc[:, ~test_eco_merge.columns.duplicated()]

# Remove duplicates from train_df and test_df before concatenation
train_df = train_df.loc[:, ~train_df.columns.duplicated()]
test_df = test_df.loc[:, ~test_df.columns.duplicated()]

# Concatenate the merged economic indicators with the original train and test dataframes
train_df = pd.concat([train_df, train_eco_merge], axis=1) 
test_df = pd.concat([test_df, test_eco_merge], axis=1)

# Remove any potential duplicates in the concatenated dataframes
train_df = train_df.loc[:, ~train_df.columns.duplicated()]
test_df = test_df.loc[:, ~test_df.columns.duplicated()]

# Print the columns to ensure there are no duplicates
print("Columns in train_df after concatenation:", train_df.columns.tolist()) 
print("Columns in test_df after concatenation:", test_df.columns.tolist())

Columns in train_df after concatenation: ['ID', 'customer_id', 'country_id', 'tbl_loan_id', 'lender_id', 'loan_type', 'Total_Amount', 'Total_Amount_to_Repay', 'duration', 'New_versus_Repeat', 'Amount_Funded_By_Lender', 'Lender_portion_Funded', 'Lender_portion_to_be_repaid', 'target', 'disbursement_date_year', 'disbursement_date_month', 'disbursement_date_day', 'due_date_year', 'due_date_month', 'due_date_day', 'Average precipitation in depth (mm per year)', 'Deposit interest rate (%)', 'Fossil fuel energy consumption (% of total)', 'Inflation, consumer prices (annual %)', 'Interest rate spread (lending rate minus deposit rate, %)', 'Lending interest rate (%)', 'Official exchange rate (LCU per US$, period average)', 'Real interest rate (%)', 'Unemployment rate']
Columns in test_df after concatenation: ['ID', 'customer_id', 'country_id', 'tbl_loan_id', 'lender_id', 'loan_type', 'Total_Amount', 'Total_Amount_to_Repay', 'duration', 'New_versus_Repeat', 'Amount_Funded_By_Lender', 'Lender_po

In [21]:
train_df = clean_column_names(train_df)
test_df = clean_column_names(test_df)
print("Final cleaned columns in train_df:", train_df.columns.tolist())
print("Final cleaned columns in test_df:", test_df.columns.tolist())

Final cleaned columns in train_df: ['ID', 'customer_id', 'country_id', 'tbl_loan_id', 'lender_id', 'loan_type', 'Total_Amount', 'Total_Amount_to_Repay', 'duration', 'New_versus_Repeat', 'Amount_Funded_By_Lender', 'Lender_portion_Funded', 'Lender_portion_to_be_repaid', 'target', 'disbursement_date_year', 'disbursement_date_month', 'disbursement_date_day', 'due_date_year', 'due_date_month', 'due_date_day', 'Average_precipitation_in_depth_mm_per_year_', 'Deposit_interest_rate_', 'Fossil_fuel_energy_consumption_of_total_', 'Inflation_consumer_prices_annual_', 'Interest_rate_spread_lending_rate_minus_deposit_rate_', 'Lending_interest_rate_', 'Official_exchange_rate_LCU_per_US_period_average_', 'Real_interest_rate_', 'Unemployment_rate']
Final cleaned columns in test_df: ['ID', 'customer_id', 'country_id', 'tbl_loan_id', 'lender_id', 'loan_type', 'Total_Amount', 'Total_Amount_to_Repay', 'duration', 'New_versus_Repeat', 'Amount_Funded_By_Lender', 'Lender_portion_Funded', 'Lender_portion_to_be

In [22]:
# Convert country_id properly
train_df['country_id'] = train_df['country_id'].apply(lambda x: x[0] if isinstance(x, tuple) else x).fillna(-1).astype(int)
test_df['country_id'] = test_df['country_id'].apply(lambda x: x[0] if isinstance(x, tuple) else x).fillna(-1).astype(int)

# Print the dtype of the 'country_id' column specifically to avoid confusion
print("train_df 'country_id' dtype after conversion:", train_df['country_id'].dtypes)  # Should be 'int32'
print("test_df 'country_id' dtype after conversion:", test_df['country_id'].dtypes)


train_df 'country_id' dtype after conversion: int32
test_df 'country_id' dtype after conversion: int32


# Feature and Target Separation for Model Training and Testing

In [24]:
# Separate features and target
X = train_df.drop(columns=['ID', 'target'])
y = train_df['target']
X_test = test_df.drop(columns=['ID'])

In [25]:
# Align the columns of the test set with the training set
X_test = X_test[X.columns]

In [26]:
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns 
print("Numerical columns:", numerical_cols.tolist())

Numerical columns: ['customer_id', 'tbl_loan_id', 'lender_id', 'Total_Amount', 'Total_Amount_to_Repay', 'duration', 'Amount_Funded_By_Lender', 'Lender_portion_Funded', 'Lender_portion_to_be_repaid', 'Average_precipitation_in_depth_mm_per_year_', 'Deposit_interest_rate_', 'Fossil_fuel_energy_consumption_of_total_', 'Inflation_consumer_prices_annual_', 'Interest_rate_spread_lending_rate_minus_deposit_rate_', 'Lending_interest_rate_', 'Official_exchange_rate_LCU_per_US_period_average_', 'Real_interest_rate_', 'Unemployment_rate']


# Scaling Numerical Features Using RobustScaler

In [28]:
scaler = RobustScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])


# Compute Inverse Class Weights for Imbalanced Dataset

In [30]:
# Compute inverse class weights based on the training labels
class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weight_dict = dict(zip(np.unique(y), class_weights))


# Stratified K-Fold Cross-Validation and LightGBM Model Training


In [32]:
# Stratified K-Fold cross-validation on the entire dataset (X, y)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# List to store F1 scores and accuracy scores for each fold
f1_scores = []
accuracy_scores = []

# Cross-validation loop
for fold, (train_index, val_index) in enumerate(skf.split(X, y), start=1):
    X_train_fold, X_val_fold = X.iloc[train_index], X.iloc[val_index]
    y_train_fold, y_val_fold = y.iloc[train_index], y.iloc[val_index]

    # Compute sample weights based on class weights
    sample_weights = y_train_fold.map(class_weight_dict)
    
    # Create dataset for LightGBM, including sample weights
    train_data = lgb.Dataset(X_train_fold, label=y_train_fold, weight=sample_weights)
    valid_data = lgb.Dataset(X_val_fold, label=y_val_fold, reference=train_data)
    
    # Define model parameters (remove class_weight from params)
    params = {
        'objective': 'binary',
        'random_state': 42,
        'learning_rate': 0.05,
        'lambda_l2': 1.0,
        'num_leaves': 81,  # Number of leaves per tree
        'min_child_samples': 30,
        'max_depth': 13
    }
    
    # Define the early stopping callback
    early_stopping_callback = lgb.callback.early_stopping(stopping_rounds=80, verbose=True)
    
    # Train the LightGBM model on the current fold with early stopping
    model = lgb.train(
        params,
        train_data,
        num_boost_round=3000,
        valid_sets=[valid_data],
        valid_names=['validation'],
        callbacks=[early_stopping_callback]  # Use the callback here
    )
    
    # Make predictions on the validation set
    y_pred = (model.predict(X_val_fold, num_iteration=model.best_iteration) > 0.5).astype(int)
    
    # Evaluate the model using F1 score and accuracy
    f1_score_val = f1_score(y_val_fold, y_pred, average='weighted')
    accuracy_score_val = accuracy_score(y_val_fold, y_pred)
    
    f1_scores.append(f1_score_val)
    accuracy_scores.append(accuracy_score_val)

    # Print out the classification report for each fold
    print(f"Classification Report for Fold {fold}:")
    print(classification_report(y_val_fold, y_pred))
    print(f"F1 Score for Fold {fold}: {f1_score_val:.4f}")
    print(f"Accuracy for Fold {fold}: {accuracy_score_val:.4f}\n")

# Print average F1 score and accuracy across all folds
print("Average F1 Score across all folds:", np.mean(f1_scores))
print("Average Accuracy across all folds:", np.mean(accuracy_scores))

# Train the final LightGBM model on the full dataset (X, y) with early stopping
train_data_full = lgb.Dataset(X, label=y, weight=y.map(class_weight_dict))
model_full = lgb.train(
    params,
    train_data_full,
    num_boost_round=3000,
    valid_sets=[train_data_full],
    valid_names=['train'],
    callbacks=[early_stopping_callback]  # Use the callback for early stopping here
)

# Make predictions on the test set
test_predictions = (model_full.predict(X_test, num_iteration=model_full.best_iteration) > 0.5).astype(int)

[LightGBM] [Info] Number of positive: 1007, number of negative: 53916
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006632 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1957
[LightGBM] [Info] Number of data points in the train set: 54923, number of used features: 17
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500153 -> initscore=0.000611
[LightGBM] [Info] Start training from score 0.000611
Training until validation scores don't improve for 80 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM

In [33]:
print(f"Number of test predictions: {len(test_predictions)}")
print(f"Number of rows in test_df: {len(test_df)}")

Number of test predictions: 68654
Number of rows in test_df: 68654


# Save Predictions to Submission CSV

In [35]:
submission_df = test_df[['ID']].copy()  # Keep 'farmid' for submission
submission_df['Target'] = test_predictions
print(f"Number of rows in submission_df: {submission_df.shape[0]}")

Number of rows in submission_df: 68654


In [36]:
#Save the results to a CSV file 
submission_df.to_csv('submission.csv', index=False)

print("Submission file 'submission.csv' created successfully.")

Submission file 'submission.csv' created successfully.


In [37]:
from IPython.display import HTML 
HTML('<a href="submission.csv" download>Download submission.csv</a>')